In [1]:
import numpy as np


Следуя методу Гаусса, напишем функцию, принимающую линейную систему и решающую её.

In [38]:
def solve_lin_system(A, b):
    n = A.shape[0]
    U = np.copy(A)
    
    L = np.zeros((n, n))
    for i in range(n):
        L[i][i] = 1   
    P = np.copy(L)
    
    def _fill_ULP(U, L, P, i):
        rest = np.copy(U[i, i:n])
        U[i, i:n] = np.copy(U[index, i:n])
        U[index, i:n] = np.copy(rest)
        
        rest = np.copy(L[i, :i])
        L[i, 0:i] = np.copy(L[index, 0:i])
        L[index, 0:i] = np.copy(rest)
        
        rest = np.copy(P[i, :])
        P[i, :] = np.copy(P[index, :])
        P[index, :] = np.copy(rest)
        
        return U, L, P
    
    for i in range(n - 1):
        index = np.argmax(np.abs(U[i:n, i])) + i
        U, L, P = _fill_ULP(U, L, P, i)
        
        for j in range(i + 1, n):
            L[j, i] = U[j, i] / U[i, i]
            U[j, i:n] = U[j, i:n] - L[j, i] * U[i, i:n]
    
    next_b = np.zeros(n)
    
    for i in range(n):
        for j in range(n):
            if (P[i, j] == 1):
                next_b[i] = b[j]
                
    y = np.zeros(n)
    for i in range(n):
        y[i] = next_b[i]
        for j in range(i):
            y[i] -= y[j] * L[i][j]
        y[i] /= L[i][i]
        
    # print(next_b)
    # print(y)

    x = np.zeros(n)
    for i in range(n - 1, -1, -1):
        x[i] = y[i]
        for j in range(n - 1, i, -1):
            x[i] -= x[j] * U[i][j]
        x[i] /= U[i][i]
    
    return U, L, P, x


Сравним наше решение с аналогичным в np.linalg. 

Для этого будем генерировать матрицы заданного размера и вычислять евклидову норму разности нашего решения и из numpy. 


In [41]:
def compare_my_solution_and_numpy(matrix_size):
    print(f'############### matrix_size == {matrix_size} ###############', end='\n\n')
    
    A = np.random.rand(matrix_size, matrix_size)
    b = np.random.rand(matrix_size)
    
    print(b)
    
    U, L, P, result = solve_lin_system(A, b)
    # print(U, L, P)
    
    np_out = np.linalg.solve(A, b)
    
    print(f'solve_linear_system(A, b) == {result}')
    print(f'np.linalg.solve(A, b) == {np_out}')
    print("np.linalg.norm(result - np_out, 2) == %s" % str(np.linalg.norm(result - np_out, 2)))
    print()
    

In [42]:
compare_my_solution_and_numpy(3)
compare_my_solution_and_numpy(5)
compare_my_solution_and_numpy(10)
compare_my_solution_and_numpy(30)

############### matrix_size == 3 ###############

[0.11062729 0.63899119 0.3084424 ]
solve_linear_system(A, b) == [-6.0418092   1.40882723  8.46030528]
np.linalg.solve(A, b) == [-6.0418092   1.40882723  8.46030528]
np.linalg.norm(result - np_out, 2) == 0.0

############### matrix_size == 5 ###############

[0.75928687 0.18803727 0.25512291 0.43922731 0.28306755]
solve_linear_system(A, b) == [ 0.10085169 -0.17294323  0.34774807 -0.62455854  0.94787639]
np.linalg.solve(A, b) == [ 0.10085169 -0.17294323  0.34774807 -0.62455854  0.94787639]
np.linalg.norm(result - np_out, 2) == 2.2073972817097756e-16

############### matrix_size == 10 ###############

[0.36432431 0.05155039 0.0096095  0.22345004 0.53374594 0.65224212
 0.62879261 0.46476916 0.23836978 0.4176991 ]
solve_linear_system(A, b) == [ 0.20356314 -0.01010428 -0.43675749  0.32061358  0.92688616  1.033931
 -2.07773965  0.52793287 -0.88080982  0.84506885]
np.linalg.solve(A, b) == [ 0.20356314 -0.01010428 -0.43675749  0.32061358  0.9268